In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/8

In [2]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-35-87-208-109.us-west-2.compute.amazonaws.com:8000/")

In [3]:
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://mlflow-bucket-damini/543202711501021658', creation_time=1759293463518, experiment_id='543202711501021658', last_update_time=1759293463518, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 6.2 MB/s eta 0:00:00


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
import mlflow
import mlflow.sklearn
import optuna

In [7]:
from google.colab import files

uploaded = files.upload()

Saving reddit_preprocessing.xls to reddit_preprocessing.xls


In [8]:
df = pd.read_csv('reddit_preprocessing.xls')
df.head()

,clean_comment,category
0,family mormon have never tried explain them th...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [9]:
df.shape

(36607, 2)

In [12]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])
X_train,X_test,y_train,y_test= train_test_split(df['clean_comment'],df['category'],random_state=42,test_size=0.2,stratify=df['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec=vectorizer.fit_transform(X_train)
X_test_vec=vectorizer.transform(X_test)

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# Step 5: Train-test split
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train_resampled, X_test_vec, y_train_resampled, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train_resampled, y_train_resampled)
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_resampled, y_train_resampled).predict(X_test_vec))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=5)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment for SVM
run_optuna_experiment()



[I 2025-10-02 02:31:34,255] A new study created in memory with name: no-name-b7112cec-1f58-4879-8826-89839fdc4d17
[I 2025-10-02 02:33:45,033] Trial 0 finished with value: 0.8262555066079296 and parameters: {'C': 0.3308148125482614, 'kernel': 'poly'}. Best is trial 0 with value: 0.8262555066079296.
[I 2025-10-02 02:35:26,469] Trial 1 finished with value: 0.6096916299559472 and parameters: {'C': 0.00041927156320371064, 'kernel': 'poly'}. Best is trial 0 with value: 0.8262555066079296.
[I 2025-10-02 02:37:01,142] Trial 2 finished with value: 0.6361233480176212 and parameters: {'C': 0.008249605963622673, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8262555066079296.
[I 2025-10-02 02:38:32,936] Trial 3 finished with value: 0.6597356828193832 and parameters: {'C': 0.012383149950321385, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8262555066079296.
[I 2025-10-02 02:39:28,860] Trial 4 finished with value: 0.905022026431718 and parameters: {'C': 0.09027536211269001, 'kernel': 'linear'}. B

🏃 View run SVM_SMOTE_TFIDF_Trigrams at: http://ec2-35-87-208-109.us-west-2.compute.amazonaws.com:8000/#/experiments/543202711501021658/runs/d12966776d5e40b2a1185d665982213d
🧪 View experiment at: http://ec2-35-87-208-109.us-west-2.compute.amazonaws.com:8000/#/experiments/543202711501021658
